<a href="https://colab.research.google.com/github/javier-jaime/Tool-Crib/blob/master/Operations%20Research/Gurobi_ML_example_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Usage Example

In this page, we provide a simple example of using the Gurobi Machine
Learning package.

The example is entirely abstract. Its aim is only to illustrate the
basic functionalities of the package in the most simple way. For some
more realistic applications, please refer to the notebooks in the
[examples](../examples.rst)_ section.

Before proceeding to the example itself, we need to import a number of
packages. Here, we will use Scikit-learn to train regression models. We
generate random data for the regression using the
[make_regression](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html)_
function. For the regression model, we use a [multi-layer perceptron
regressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)_
neural network. We import the corresponding objects.


In [6]:
!pip install gurobipy gurobi-machinelearning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 1.9 MB/s eta 0:00:00


In [7]:
import gurobipy as gp
import numpy as np
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor

from gurobi_ml import add_predictor_constr

Certainly, we need gurobipy to build an optimization model and from the
gurobi_ml package we need the
[add_predictor_constr](../auto_generated/gurobi_ml.add_predictor_constr.rst)_
function. We also need numpy.




We start by building artificial data to train our regressions. To do so,
we use *make_regression* to obtain data with 10 features.




In [8]:
X, y = make_regression(n_features=10, noise=1.0)

Now, create the *MLPRegressor* object and fit it.




In [9]:
nn = MLPRegressor([20] * 2, max_iter=10000, random_state=1)

nn.fit(X, y)

MLPRegressor(hidden_layer_sizes=[20, 20], max_iter=10000, random_state=1)

We now turn to the optimization model. In the spirit of adversarial
machine learning examples, we use some training examples. We pick
$n$ training examples randomly. For each of the examples, we want
to find an input that is in a small neighborhood of it that leads to the
output that is closer to $0$ with the regression.

Denoting by $X^E$ our set of examples and by $g$ the
prediction function of our regression model, our optimization problem
reads:

\begin{align}\begin{aligned}
   &\min \sum_{i=1}^n y_i^2 \\
   &\text{s.t.:}\\
   &y_i = g(X_i) & & i = 1, \ldots, n,\\
   &X^E - \delta \leq X \leq X^E + \delta,\\
   \end{aligned}\end{align}

where $X$ is a matrix of variables of dimension
$n \times 10$ (the number of examples we consider and number of
features in the regression respectively), $y$ is a vector of free
(unbounded) variables and $\delta$ a small positive constant.

First, let’s pick randomly 2 training examples using numpy, and create
our gurobipy model.




In [10]:
n = 2
index = np.random.choice(X.shape[0], n, replace=False)
X_examples = X[index, :]
y_examples = y[index]

m = gp.Model()

Restricted license - for non-production use only - expires 2025-11-24


Our only decision variables in this case, are the five inputs and
outputs for the regression. We use ``gurobipy.MVar`` matrix variables
that are most convenient in this case.

The input variables have the same shape as ``X_examples``. Their lower
bound is ``X_examples - delta`` and their upper bound
``X_examples + delta``.

The output variables have the shape of ``y_examples`` and are unbounded.
By default, in Gurobi variables are non-negative, we therefore need to
set an infinite lower bound.




In [11]:
input_vars = m.addMVar(X_examples.shape, lb=X_examples - 0.2, ub=X_examples + 0.2)
output_vars = m.addMVar(y_examples.shape, lb=-gp.GRB.INFINITY)

The constraints linking ``input_vars`` and ``output_vars`` can now be
added with the function
:func:`add_predictor_constr <gurobi_ml.add_predictor_constr.rst>`.

Note that because of the shape of the variables this will add the 5
different constraints.

The function returns an instance of a :class:`modeling
object <gurobi_ml.modeling.base_predictor_constr.AbstractPredictorConstr>`
that we can use later on.




In [12]:
pred_constr = add_predictor_constr(m, nn, input_vars, output_vars)

The method
:func:`print_stats <gurobi_ml.modeling.base_predictor_constr.AbstractPredictorConstr.print_stats>`
of the modeling object outputs the details of the regression model that
was added to the Gurobi.




In [13]:
pred_constr.print_stats()

Model for mlpregressor:
160 variables
82 constraints
80 general constraints
Input has shape (2, 10)
Output has shape (2, 1)

--------------------------------------------------------------------------------
Layer           Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
dense                (2, 20)           80           40            0           40 (relu)

dense0               (2, 20)           80           40            0           40 (relu)

dense1                (2, 1)            0            2            0            0

--------------------------------------------------------------------------------


To finish the model, we set the objective, and then we can optimize it.




In [14]:
m.setObjective(output_vars @ output_vars, gp.GRB.MINIMIZE)

m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 82 rows, 182 columns and 1322 nonzeros
Model fingerprint: 0xa25d70ec
Model has 2 quadratic objective terms
Model has 80 general constraints
Variable types: 182 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-03, 3e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e-02, 2e+00]
  RHS range        [4e-02, 1e+00]
Presolve removed 26 rows and 116 columns
Presolve time: 0.05s
Presolved: 56 rows, 66 columns, 393 nonzeros
Presolved model has 1 quadratic objective terms
Variable types: 50 continuous, 16 integer (16 binary)

Root relaxation: objective 4.343798e+04, 73 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds  

The method
:func:`get_error <gurobi_ml.modeling.base_predictor_constr.AbstractPredictorConstr.get_error>`
is useful to check that the solution computed by Gurobi is correct with
respect to the regression model we use.

Let $(\bar X, \bar y)$ be the values of the input and output
variables in the computed solution. The function returns
$g(\bar X) - y$ using the original regression object.

Normally, all values should be small and below Gurobi’s tolerances in this example.




In [15]:
pred_constr.get_error()

array([[5.68434189e-14],
       [2.10942375e-15]])

We can look at the computed values for the output variables and compare
them with the original target values.




In [16]:
print("Computed values")
print(pred_constr.output_values.flatten())

Computed values
[208.41780107   0.        ]


In [17]:
print("Original values")
print(y_examples)

Original values
[288.07664021 -35.1215552 ]


Finally, we can remove ``pred_constr`` with the method
:func:`remove() <gurobi_ml.modeling.base_predictor_constr.AbstractPredictorConstr.remove>`.




In [18]:
pred_constr.remove()